In [1]:
import torch
from torch import sigmoid
from torch.nn import Linear, BCEWithLogitsLoss
from torch.optim import Adam
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
#from torch_geometric.data import DataLoader
from torch_geometric.loader import DataLoader
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
dataset = TUDataset(root='../data/', name='MUTAG')

In [3]:
embedding_size = 32
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        
        #GCN Layers
        self.input = GCNConv(dataset.num_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)
        self.output = Linear(embedding_size*2, 1)
        
    def forward(self, x, edge_index, batch_index):
        out = self.input(x, edge_index)
        out = F.relu(out)
        out = self.conv1(out, edge_index)
        out = F.relu(out)
        out = self.conv2(out, edge_index)
        out = F.relu(out)
        out = self.conv3(out, edge_index)
        out = F.relu(out)
        
        global_pool = torch.cat([gmp(out, batch_index), gap(out, batch_index)], dim=1)
        
        out = self.output(global_pool)
        
        return out, global_pool

In [4]:
model = GCN()
print(model)

GCN(
  (input): GCNConv(7, 32)
  (conv1): GCNConv(32, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 32)
  (output): Linear(in_features=64, out_features=1, bias=True)
)


In [5]:
loss_fn = BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=0.001)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

GRAPH_BATCH_SIZE = 8
train_loader = DataLoader(dataset[:int(len(dataset)*0.8)], batch_size=GRAPH_BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset[int(len(dataset)*0.8):], batch_size=GRAPH_BATCH_SIZE, shuffle=True)

In [10]:
def train_model(data):
    for batch in data:
        batch.to(device)
        optimizer.zero_grad()
        prediction, embedding = model(batch.x.float(), batch.edge_index, batch.batch)
        prediction = sigmoid(prediction).round()
        batch.y = batch.y.unsqueeze(1).float()
        loss = loss_fn(prediction, batch.y)
        loss.backward()
        optimizer.step()
    return loss, embedding

In [11]:
losses = []
for epoch in range(1, 101):
    loss, embedding = train_model(train_loader)
    losses.append(loss)
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch} | Train Loss: {loss:.3f}")

Epoch 1 | Train Loss: 0.647
Epoch 10 | Train Loss: 0.543
Epoch 20 | Train Loss: 0.710
Epoch 30 | Train Loss: 0.567
Epoch 40 | Train Loss: 0.837
Epoch 50 | Train Loss: 0.543
Epoch 60 | Train Loss: 0.773
Epoch 70 | Train Loss: 0.503
Epoch 80 | Train Loss: 0.440
Epoch 90 | Train Loss: 0.773
Epoch 100 | Train Loss: 0.773
